# Expectation-Maximization Algorithm

Finally, we get to the most renowned optimizatio algorithm for HMMs: The Baum-Welch optimization/reparameterization algorithm.  Ultimately, this algorithm is an early-implemetaion of the relatively broad class of expectation-maximization algorithms.

Ultimately, the difference between the EM algorithm that we discuss here, and previous likelihood-maximizing algorithms is the definition of the 'data' that is used in the likelihood function.  Specifically, the previous likelihood function that we have maximized takes the general form $\mathcal{L}(\theta | Y^T)$ effectively taking the set of observed quantitites as the set of data for the optimization. However, another apporach considers the joint set of $Y$ *and* $X$ as the full set of data, and therefore effectively maximizes the likelihood $\mathcal{L}(\theta | X^T, Y^T)$. However, the issue is that this likelihood function can no longer be explicitly maximized, becuase we dont directly observe the hidden state sequence $X^T$.  This is why we have to use the EM algorithm (known as the Baum-Welch algorithm in the context of HMMs).

Ultimately, this algorithm is going to give us an iterative method of updating the elements of the transition ($\boldsymbol{A}$) and observation ($\boldsymbol{B}$) matrices, in such a way that they will converge towards the minimum of the so-called *Complete Data Log Likelihood* $\mathcal{L}(\theta | X^T, Y^T)$.  Here, the difficult part is inferring the elements of the transition matrix $\boldsymbol{A}$, which quantifies the transition rates between hidden states. If we were able to observe these states directly, then our best estimate of the $j\to i$ transition rate $p(x_{i, t+1} | x_{t, j})$ would be

$$ \hat{A}_{ij} =  \frac{N_{j \to i}}{N_j} $$

Which, we can restate as a probabilistic equation as

$$ \hat{A}_{ij} = \frac{\sum_t p(x_{i, t}, x_{j, t-1} | Y^T)}{\sum_t p(x_{j, t-1} | Y^T)} $$

In fact, this exact equation is what the `Maximization' (M) step of the EM algorithm is doing in this case. So our goal is then first to figure out how to calculate the unknown probability terms on teh RHS above.  First, note that the term in the denominator is just the Bayesan probabiliy estimate, which we alreay know how to calcualte useing the Bayesian smoothing algorithm. The top term, however, is more tricky to determine.

To start, note that we can use the law of total probability to split the joint probability into a product:

$$ p(x_{i, t}, x_{j, t-1} | Y^T) = p(x_{i, t} | x_{j, t-1}, Y^T) p(x_{j, t-1} | Y^T) $$

where again, the second term on the RHS is the Bayesian smoothed state estimate.

Now, to deal with the first toer on the RHS of the equation above, we first make use of Bayes rule, which states for random variables $X$ and $Y$ that

$$ P(X|Y) = \frac{P(Y|X)P(X)}{\sum_x P(Y|X)P(X)} $$

and therefore we can rewrite the above probability term as

$$ p(x_{i, t} | x_{j, t-1}, Y^T) = \frac{p(Y^T | x_{i, t}, x_{j, t-1})p(x_{i, t} | x_{j, t-1})}{\sum_{i} p(Y^T | x_{i, t}, x_{j, t-1})p(x_{i, t} | x_{j, t-1}) } $$

Now, we can splmplify this a bit by noting that $p(x_{i, t} | x_{j, t-1}) = A_{ij}$.  Next, we make use of a property of HMMs known as conditional independence of observations, given the hiden states, which effectively means that, for a sequence of three hidden states, $x_1, x_2, x_3$ and corresponding observations $y_1, y_2, y_3$, we can split a conditional joint probability as

$$ p(y_1, y_2, y_3 | x_1, x_2, x_3) = p(y_1 | x_1)p(y_2 | x_2)p(y_3 | x_3) $$

Using this logic we can rewrite the first term in the RHS numerator above as

$$
p(Y^T | x_{i, t} x_{j, t-1}) = p(Y^{[0, t-1]}, Y^{[t, T]} | x_{i, t}, x_{j, t-1})  \\
\, \\
= p(Y^{[0, t-1]} | x_{j, t-1})p(Y^{[t, T]} | x_{i, t}) \\
\, \\
= p(Y^{[0, t-1]} | x_{j, t-1})p(Y_t | x_{i,t})p(Y^{[t+1, T]} | x_{i, t}) \\
\, \\
= \alpha_{t-1}(j) B_{i, Y_t} \beta_t(j)
$$

and therefore we can rewrite our final expression for the joint probability as

$$ p(x_{i, t}, x_{j, t-1} | Y^T) = \frac{ \alpha_{t-1}(j)\beta_t(i) B_{i, y_t} A_{ij} p(x_{j, t-1} | Y^T) }{\sum_i \alpha_{t-1}(j)\beta_t(i) B_{i, y_t} A_{ij} } $$

And therefore we can estimate the terms in the $A$ matrix as

$$ \hat{A}_{ij} = \sum_t\left[\frac{\alpha_{t-1}(j)\beta_t(i) B_{i, y_t} A_{ij} p(x_{j, t-1} | Y^T) }{ \sum_i \alpha_{t-1}(j)\beta_t(i) B_{i, y_t} A_{ij} }\right] \frac{1}{\sum_t p(x_{j, t-1} | Y^T)} $$

The subtle thing here, is that in order to calcualte the terms on the RHS, we need to have an estiamte already for $\boldsymbol{A}$. This is ultimately where the iterative nature of the EM algorithm enters the picture.

To start, we will go through this calculation and show how to run it using the `hidden` package before discussing the similar calcualte for the observation matrix $\boldsymbol{B}$ (which turns out to be much simpler), and then finally more on to the actual implementation of the EM algorithm.


In [1]:
import os
from pathlib import Path
import numpy as np
from hidden import dynamics, infer
from hidden.filters import bayesian

hmm = dynamics.HMM(2, 2)
hmm.init_uniform_cycle()
hmm.run_dynamics(200)

obs_ts = hmm.get_obs_ts()

analyzer = infer.MarkovInfer(2, 2)

# Now start off with an estimate of the A and B matrices that is close to the true version
A_perturb = np.array([
    [0.1, 0.05],
    [-0.1, -0.05]
])

B_perturb = np.array([
    [0.06, -0.05],
    [-0.06, 0.05]
])

A_init = hmm.A + A_perturb
B_init = hmm.B + B_perturb

In [2]:
# Now we can start the calculation.  So, for this we need the bayesian filtered
# probabilities, the alpha terms, and the beta terms

analyzer.bayesian_smooth(obs_ts, A_init, B_init)
analyzer.alpha(obs_ts, A_init, B_init, norm=True)
analyzer.beta(obs_ts, A_init, B_init, norm=True)

alpha_norm = analyzer.alpha_tracker
beta_norm = analyzer.beta_tracker
bayes = analyzer.bayes_smooth

In [3]:
# Now to get a single term (for a given time value), we can go over the
# calcualtion element-wise

# start with the 00 and 10 terms, which are for the 0 -> 0 and 1 -> 0 transitions

sample_time = 10

def get_numerator(i, j):
    return (
        A_init[i, j] * B_init[obs_ts[sample_time], i]
        * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, i]
        * bayes[sample_time - 1, j]
    )

def get_denom(j):
    return (
        (
            A_init[0, j] * B_init[obs_ts[sample_time], 0]
            * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, 0]
        )
        + (
            A_init[1, j] * B_init[obs_ts[sample_time], 1]
            * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, 1]
        )
    )


numerator_00 = get_numerator(0, 0)
numerator_10 = get_numerator(1, 0)
numerator_01 = get_numerator(0, 1)
numerator_11 = get_numerator(1, 1)

# Now, the denominators are the same for both i values (across a column)

# Used for A_00 and A_10
denom_0 = get_denom(0)
# Used for A_01 and A_11
denom_1 = get_denom(1)

# Now, if we only had this single point in time, then we can calcualte the entries

A_00_new = (numerator_00 / denom_0) * (1 / bayes[sample_time - 1, 0])
A_10_new = (numerator_10 / denom_0) * (1 / bayes[sample_time - 1, 0])
A_01_new = (numerator_01 / denom_1) * (1 / bayes[sample_time - 1, 1])
A_11_new = (numerator_11 / denom_1) * (1 / bayes[sample_time - 1, 1])

A_new = np.array([
    [A_00_new, A_01_new],
    [A_10_new, A_11_new]
])

In [4]:
A_new

array([[0.96916984, 0.80885895],
       [0.03083016, 0.19114105]])

In [26]:
# And to see if it is still correctly normalized
A_new.sum(axis=0)

array([1., 1.])

In [27]:
# Perfect.
# So, now we can turn this into a procedure which will sum up observations over
# the entire time series of observations

# First we calculate the alpha, beta, and bayesian estimates
analyzer.bayesian_smooth(obs_ts, A_init, B_init)
analyzer.alpha(obs_ts, A_init, B_init, norm=True)
analyzer.beta(obs_ts, A_init, B_init, norm=True)

alpha_norm = analyzer.alpha_tracker
beta_norm = analyzer.beta_tracker
bayes = analyzer.bayes_smooth

# Then we calculate the numeratiors and denominators in the same manner as above
def get_numerator(i, j, sample_time):
    return (
        A_init[i, j] * B_init[i, obs_ts[sample_time]]
        * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, i]
        * bayes[sample_time - 1, j]
    )

def get_denom(j, sample_time):
    return (
        (
            A_init[0, j] * B_init[0, obs_ts[sample_time]]
            * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, 0]
        )
        + (
            A_init[1, j] * B_init[1, obs_ts[sample_time]]
            * alpha_norm[sample_time - 1, j] * beta_norm[sample_time, 1]
        )
    )


numer_00 = np.zeros(len(obs_ts) - 1)
numer_10 = np.zeros_like(numer_00)
numer_01 = np.zeros_like(numer_00)
numer_11 = np.zeros_like(numer_00)

denom_0 = np.zeros_like(numer_00)
denom_1 = np.zeros_like(numer_00)

for t in range(1, len(obs_ts)):
    numer_00[t-1] = get_numerator(0, 0, t)
    numer_10[t-1] = get_numerator(1, 0, t)
    numer_01[t-1] = get_numerator(0, 1, t)
    numer_11[t-1] = get_numerator(1, 1, t)

    denom_0[t-1] = get_denom(0, t)
    denom_1[t-1] = get_denom(1, t)

A_00_new = sum(numer_00 / denom_0) * (1 / sum(bayes[:-1, 0]))
A_10_new = sum(numer_10 / denom_0) * (1 / sum(bayes[:-1, 0]))
A_01_new = sum(numer_01 / denom_1) * (1 / sum(bayes[:-1, 1]))
A_11_new = sum(numer_11 / denom_1) * (1 / sum(bayes[:-1, 1]))
    
A_00_new_partial = sum(numer_00 / denom_0)
A_10_new_partial = sum(numer_10 / denom_0)
A_01_new_partial = sum(numer_01 / denom_1)
A_11_new_partial = sum(numer_11 / denom_1)

bayes_00_part = (1 / sum(bayes[:-1, 0]))
bayes_10_part = (1 / sum(bayes[:-1, 0]))
bayes_01_part = (1 / sum(bayes[:-1, 1]))
bayes_11_part = (1 / sum(bayes[:-1, 1]))

A_new = np.array([
    [A_00_new, A_01_new],
    [A_10_new, A_11_new]
])


In [28]:
bayes.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [29]:
A_new, A_new.sum(axis=0)

(array([[0.76114956, 0.30559352],
        [0.23885044, 0.69440648]]),
 array([1., 1.]))

In [77]:
np.vstack([B_init[0, :], B_init[0, :]])

array([[0.96, 0.05],
       [0.96, 0.05]])

In [110]:
# Now, we can clean this up a bit by vectorizing some of these calculations,
# specifically, we organize all of the numerators into a matrix 'xi', we cant
# really vectorize over time, but I think we can on a per-time-point basis

def get_xi_matrix(obs, A, B, alpha_norm, beta_norm, bayes):
    xi = np.zeros((2, 2, len(obs_ts) - 1))

    for t in range(1, len(obs_ts)):
        xi[:, :, t - 1] = (
            A
            * np.vstack([B[obs[t], :], B[obs[t], :]]).T
            # * np.hstack([B[obs[t], :], B[obs[t], :]])
            * np.outer(
                beta_norm[t, :], (alpha_norm[t - 1, :] * bayes[t - 1, :])
            )
        )

    return xi

# The denominator for each element is just a column-wize normalization without
# the bayes factor, so the calcualtion is very similar
def get_denom_matrix(obs, A, B, alpha_norm, beta_norm):
    xi_denom = np.zeros((1, 2, len(obs_ts) - 1))

    for t in range(1, len(obs_ts)):
        xi_denom[:, :, t-1] = np.sum(
            A
            # * np.repeat(
                # B[obs[t], :].reshape(1, A.shape[0]),
                # 2, axis=0
            # )
            * np.vstack([B[obs[t], :], B[obs[t], :]]).T
            * np.outer(
                beta_norm[t, :], alpha_norm[t - 1, :]
            )
        , axis=0)

    # And then we sum over the columns
    xi_denom = np.vstack([xi_denom, xi_denom])

    return xi_denom

xi_num = get_xi_matrix(obs_ts, A_init, B_init, alpha_norm, beta_norm, bayes)
xi_denom = get_denom_matrix(obs_ts, A_init, B_init, alpha_norm, beta_norm)

In [106]:
xi_num.shape, xi_denom.shape

((2, 2, 199), (2, 2, 199))

In [107]:
xi_num[:, :, 9]

array([[8.27925203e-05, 4.62673302e-03],
       [1.48795671e-03, 6.17701900e-01]])

In [108]:
xi_num[:, :, 9]

array([[8.27925203e-05, 4.62673302e-03],
       [1.48795671e-03, 6.17701900e-01]])

In [109]:
get_numerator(0, 0, 10), get_numerator(1, 1, 10), get_numerator(0, 1, 10), get_numerator(1, 0, 10)

(8.279252026599447e-05,
 0.6177019002280281,
 0.004626733021507513,
 0.0014879567121813487)

In [111]:
xi_denom[:, :, 9]

array([[0.02251799, 0.39778689],
       [0.02251799, 0.39778689]])

In [112]:
get_denom(0, 10), get_denom(1, 10)


(0.0225179929378272, 0.39778688513214056)

In [30]:
get_denom(0, 10), get_denom(1, 10)

(0.2334544270044038, 0.0013099873226974951)

In [113]:
# Now, we should be able to do an element-wise division of these matrices,
# to get the series of numerical values for each matrix entry
ratio = np.divide(xi_num, xi_denom)
ratio.shape


(2, 2, 199)

In [114]:
ratio[:,  :, 10]

array([[1.03849123e-03, 1.52472458e-02],
       [1.50652224e-02, 1.64312053e+00]])

In [124]:
np.sum(ratio, axis=2)

array([[84.24136389, 46.63270354],
       [31.83783815, 85.85199759]])

In [123]:
A_00_new_partial, A_01_new_partial, A_10_new_partial, A_11_new_partial

(84.24136388656765, 46.63270354203865, 31.83783815140865, 85.85199758862484)

In [161]:
# And then we just need to get the bayes matrix that we end up multiplying this
# all by
bayes_matrix = np.repeat((1 / bayes[:-1, :].sum(axis=0)).reshape(1, 2), 2, axis=0)
bayes_matrix

array([[0.00861481, 0.00754804],
       [0.00861481, 0.00754804]])

In [162]:
bayes_00_part, bayes_01_part

(0.008614807669618899, 0.007548041332061026)

In [163]:
# So, we should now be able to calcualte the whole updated matrix by performing
# operations on the xi values

A_new = np.sum(ratio, axis=2) * (bayes_matrix)
A_new

array([[0.72572315, 0.35198557],
       [0.27427685, 0.64801443]])

In [164]:
A_new.sum(axis=0)

array([1., 1.])

In [165]:
# So this looks like it is working,

# HOWEVER the actual bayesian filter does not appear to be working anymore... for some reason...
# so ill dig into that now, but we can quickly patch together a v1 implementation of the 'update_a_matrix' routine

def update_A_matrix(obs_ts, analyzer: infer.MarkovInfer, A_current, B_current):
    analyzer.alpha(obs_ts, A_current, B_current, norm=True)
    analyzer.beta(obs_ts, A_current, B_current, norm=True)
    analyzer.bayesian_smooth(obs_ts, A_current, B_current)

    xi_num = get_xi_matrix(
        obs_ts, A_current, B_current,
        analyzer.alpha_tracker,
        analyzer.beta_tracker,
        analyzer.bayes_smooth
    )
    xi_denom = get_denom_matrix(
        obs_ts,
        A_current,
        B_current,
        analyzer.alpha_tracker,
        analyzer.beta_tracker
    )

    ratio = xi_num / xi_denom

    bayes_matrix = np.repeat(
        (1 / analyzer.bayes_smooth[:-1, :].sum(axis=0)).reshape(1, 2), 2,
        axis=0
    )

    new_A_matrix = np.sum(ratio, axis=2) * bayes_matrix
    return new_A_matrix

In [166]:
# And as a test to be sure this gets the same result

update_A_matrix(obs_ts, analyzer, A_init, B_init)


array([[0.72572315, 0.35198557],
       [0.27427685, 0.64801443]])

In [167]:
A_new

array([[0.72572315, 0.35198557],
       [0.27427685, 0.64801443]])